# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [2]:
# %pip install optuna
# %pip install belashovplot
# %pip install timm

# Импорты и настройка модулей

In [3]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.training import train, confusion
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
from pickle import dump
import torch
import numpy
import timm
import optuna
import pandas

C:\Users\uclap\Documents\Coding\Python\OpticalEncoder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FigureWidthHeight = (16, 16)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA GeForce RTX 3060 Ti


# Инициализация вспомогательных методов

## Комбинированная лосс функция

In [6]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

## Электронная моель ResNet

In [7]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=True,  in_chans=1, num_classes=classes).to(device)

## Инициализация системы по полному набору параметров

In [8]:
def initialize(
    wavelength:float, N:int, length:float, pixels:int, masks_amount:int, distance:float,
    spatial_coherence:float, time_coherence:float, time_relaxation:float, mean_samples:int,
    detector_size:float, detectors_amount:int,
    loss_function_proportion:float,  batch_size:int, optimizer_type:str
):
    print("Инициализированна модель со следующими параметрами:")
    print(f"\tДлинна волны:                             {engineering(wavelength, 'м')}")
    print(f"\tКоличество вычислительных пикселей:       {N}")
    print(f"\tКоличество пикселей маски:                {pixels}")
    print(f"\tРазмер оптических элементов:              {engineering(length, 'м')}")
    print(f"\tРазмер пикселя маски:                     {engineering(length/pixels, 'м')}")
    print(f"\tРасстояние между слоями:                  {engineering(distance, 'м')}")
    
    print(f"\tВременная когерентность:                  {engineering(time_coherence, 'с')}")
    print(f"\tВремя релаксации:                         {engineering(time_relaxation, 'c')}")
    print(f"\tПространственная когерентность:           {engineering(spatial_coherence, 'м')}")
    print(f"\tКоличество усреднений:                    {mean_samples}")

    print(f"\tРазмер детекторов:                        {engineering(detector_size, 'м')}")
    print(f"\tКоличество детекторов:                    {detectors_amount} на {detectors_amount}")

    print(f"\tПропорция CE к MSE лосс:                  {loss_function_proportion}")
    print(f"\tРазмер батча:                             {batch_size}")
    print(f"\tТип оптимизатора:                         {optimizer_type}")
    print()
    
    incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length).to(device)

    spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
    detectors_filter = Gaussian((detector_size, detector_size), (0,0))
    detectors = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter).to(device)

    electronic = resnet18()

    propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
    phase_modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
    amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
    elements = [phase_modulators[0], amplitude_modulators[0]]
    for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
        elements.append(propagation)
        elements.append(phase_modulator)
        elements.append(amplitude_modulator)
    elements.append(propagation)
    chunker = CudaMemoryChunker(sub_chunks=mean_samples)
    optical = CompositeModel(*elements)
    optical.wrap(chunker)
    optical.wrap(incoherent)
    optical.to(device)
    model = HybridModel(optical, detectors, electronic).to(device)

    dataset = Dataset('CIFAR10', batch_size, N, N, torch.complex64)
    dataset.train
    dataset.test
    
    loss_function = combined_loss(loss_function_proportion)
    optimizer_types_list = {'Adam':torch.optim.Adam, 'SGD':torch.optim.SGD, 'RMSprop':torch.optim.RMSprop, 'Adagrad':torch.optim.Adagrad}
    optimizer_type = optimizer_types_list[optimizer_type]

    return model, dataset, loss_function, optimizer_type, (incoherent, chunker, optical, detectors, electronic)

# Установка не изменяемых параметров системы

In [9]:
# Предпочтительные параметры
size = 50.0E-6
near_N = 300
near_length = 5.0E-3
wavelength = 500.0E-9
detectors_amount = 24

# Параметры когерентности
spatial_coherence = 1000.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 7

In [10]:
# Вычисляемые параметры
pixels = upper_integer(near_length/size)
length = pixels * size
cppp = upper_integer(near_N * size / length)
N = int(length / size) * cppp
detector_size = length / 60

# Отображение базы данных тренировок

In [11]:
study = optuna.create_study(study_name="ID2NN", storage="sqlite:///ID2NN.db", direction='maximize', load_if_exists=True)
dataframe:pandas.DataFrame = study.trials_dataframe()
print(f"Кол-во исследований: {len(study.trials)}")

[I 2024-05-12 14:31:53,273] Using an existing study with name 'ID2NN' instead of creating a new one.


Кол-во исследований: 518


In [12]:
dataframe.nlargest(10, 'value')

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
318,318,27.78,2024-05-07 13:51:45.791768,2024-05-07 13:58:16.192132,0 days 00:06:30.400364,21,0.609088,0.080086,0.004227,3,RMSprop,COMPLETE
464,464,27.52,2024-05-08 09:44:42.080341,2024-05-08 09:51:11.168711,0 days 00:06:29.088370,22,0.609798,0.090520,0.009854,3,RMSprop,COMPLETE
170,170,27.02,2024-05-06 16:44:10.393688,2024-05-06 16:51:03.435147,0 days 00:06:53.041459,22,0.585650,0.081026,0.006971,3,RMSprop,COMPLETE
282,282,27.00,2024-05-07 07:57:43.898133,2024-05-07 08:04:11.998489,0 days 00:06:28.100356,24,0.871667,0.087308,0.006792,3,RMSprop,COMPLETE
486,486,26.84,2024-05-08 12:11:16.355451,2024-05-08 12:17:44.135148,0 days 00:06:27.779697,23,0.924804,0.084454,0.008574,3,RMSprop,COMPLETE
473,473,26.65,2024-05-08 10:44:57.743102,2024-05-08 10:51:41.548426,0 days 00:06:43.805324,23,0.947274,0.082832,0.012314,3,RMSprop,COMPLETE
328,328,26.57,2024-05-07 14:57:25.478992,2024-05-07 15:04:07.806165,0 days 00:06:42.327173,22,0.619450,0.088087,0.002438,3,RMSprop,COMPLETE
185,185,26.56,2024-05-06 18:33:01.453181,2024-05-06 18:40:32.413576,0 days 00:07:30.960395,22,0.918696,0.082408,0.003772,3,RMSprop,COMPLETE
202,202,26.47,2024-05-06 20:34:45.985106,2024-05-06 20:41:43.901028,0 days 00:06:57.915922,22,0.881049,0.099628,0.005962,3,RMSprop,COMPLETE
368,368,26.42,2024-05-07 21:54:43.933547,2024-05-07 22:01:20.587107,0 days 00:06:36.653560,22,0.989623,0.083510,0.006275,3,RMSprop,COMPLETE


In [13]:
dataframe[dataframe['value'] >= 25].nlargest(10, 'params_distance')

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
419,419,25.39,2024-05-08 04:43:19.471215,2024-05-08 04:50:06.184039,0 days 00:06:46.712824,21,0.927108,0.161614,0.006927,3,RMSprop,COMPLETE
148,148,25.66,2024-05-06 14:09:26.010912,2024-05-06 14:16:19.416841,0 days 00:06:53.405929,21,0.016943,0.136854,0.006616,3,RMSprop,COMPLETE
78,78,25.73,2024-05-06 04:14:13.851739,2024-05-06 04:22:15.120051,0 days 00:08:01.268312,20,0.046140,0.111767,0.006598,4,Adam,COMPLETE
256,256,25.32,2024-05-07 05:01:05.373643,2024-05-07 05:07:34.515358,0 days 00:06:29.141715,23,0.873216,0.108465,0.005069,3,RMSprop,COMPLETE
234,234,25.36,2024-05-07 02:27:20.326281,2024-05-07 02:34:03.267210,0 days 00:06:42.940929,22,0.834854,0.105779,0.006482,3,RMSprop,COMPLETE
238,238,26.07,2024-05-07 02:54:01.686042,2024-05-07 03:00:36.468071,0 days 00:06:34.782029,21,0.822325,0.104663,0.004283,3,RMSprop,COMPLETE
249,249,26.38,2024-05-07 04:15:40.388794,2024-05-07 04:22:08.712510,0 days 00:06:28.323716,23,0.822749,0.104230,0.007333,3,RMSprop,COMPLETE
210,210,25.71,2024-05-06 21:28:15.845935,2024-05-06 21:34:54.754249,0 days 00:06:38.908314,22,0.873626,0.103672,0.002602,3,RMSprop,COMPLETE
195,195,25.30,2024-05-06 19:45:30.912470,2024-05-06 19:52:49.290034,0 days 00:07:18.377564,23,0.877697,0.103050,0.004641,3,RMSprop,COMPLETE
182,182,25.01,2024-05-06 18:10:28.976053,2024-05-06 18:18:01.601154,0 days 00:07:32.625101,22,0.861536,0.100905,0.003497,3,RMSprop,COMPLETE


# Модель №1

In [14]:
dataframe[dataframe['number'] == 464]

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
464,464,27.52,2024-05-08 09:44:42.080341,2024-05-08 09:51:11.168711,0 days 00:06:29.088370,22,0.609798,0.09052,0.009854,3,RMSprop,COMPLETE


## Утсановка оптимизированных параметров системы

In [15]:
masks_amount = 3
batch_size = 22
distance = 0.09052
learning_rate = 0.009854
loss_function_proportion = 0.609798
optimizer_type = 'RMSprop'

In [16]:
model, dataset, loss_function, optimizer_type, (incoherent, chunker, optical, detectors, electronic) = initialize(
    wavelength, N, length, pixels, masks_amount, distance,
    spatial_coherence, time_coherence, time_relaxation, mean_samples,
    detector_size, detectors_amount,
    loss_function_proportion,  batch_size, optimizer_type
)

Инициализированна модель со следующими параметрами:
	Длинна волны:                             500.0 нм
	Количество вычислительных пикселей:       300
	Количество пикселей маски:                100
	Размер оптических элементов:              5.0 мм
	Размер пикселя маски:                     50.0 мкм
	Расстояние между слоями:                  90.52 мм
	Временная когерентность:                  10.0 нс
	Время релаксации:                         1.0 мкc
	Пространственная когерентность:           1.0 мм
	Количество усреднений:                    7
	Размер детекторов:                        83.333 мкм
	Количество детекторов:                    24 на 24
	Пропорция CE к MSE лосс:                  0.609798
	Размер батча:                             22
	Тип оптимизатора:                         RMSprop

Files already downloaded and verified
Files already downloaded and verified


## Тренировка

In [17]:
loss_histories = []
cofusion_matrixes = []

In [18]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate)
cofusion_matrixes.append(confusion(model, dataset))
print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")
for i in range(30):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:45<00:00,  9.97it/s]


Точность: 10.0%


RLoss: 1.3534687889532324, RPI1000: -0.08217206847643638: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:10<00:00,  6.14it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:39<00:00, 11.64it/s]


Точность: 22.98%


RLoss: 1.3279330910371223, RPI1000: -0.030260872185805723: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:00<00:00,  6.31it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:39<00:00, 11.39it/s]


...

Точность: 45.8%


RLoss: 0.8052864447505408, RPI1000: 0.016416877468953175: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:10<00:00,  6.14it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:38<00:00, 11.69it/s]


Точность: 45.51%


RLoss: 0.7358909594536319, RPI1000: 0.022119817515791673: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:15<00:00,  6.05it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:39<00:00, 11.53it/s]

Точность: 44.67%


In [19]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate/100)
for i in range(30):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

RLoss: 0.6234836218918457, RPI1000: -0.013484614693166601: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:08<00:00,  6.16it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:39<00:00, 11.43it/s]


Точность: 46.29%


RLoss: 0.4871749953593047, RPI1000: -0.006023195856334585: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:03<00:00,  6.25it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:40<00:00, 11.29it/s]


Точность: 46.85%


RLoss: 0.5750635869170486, RPI1000: -0.0017550955166577892: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:05<00:00,  6.21it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:40<00:00, 11.32it/s]


...

Точность: 46.06%


RLoss: 0.40594851204368326, RPI1000: -0.0028821472770355957: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:07<00:00,  6.18it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:39<00:00, 11.38it/s]


Точность: 45.69%


RLoss: 0.45668038209328055, RPI1000: 0.011949721679428274: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [06:07<00:00,  6.19it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:40<00:00, 11.23it/s]

Точность: 46.14%


In [20]:
data = (model, dataset, loss_function_proportion, optimizer_type, (incoherent, chunker, optical, detectors, electronic), loss_histories, cofusion_matrixes)
with open('Trained ID2NN HSC.pkl', 'wb') as file:
    dump(data, file)